## Load Conversations

In [1]:
# import sys
# sys.path.append('..')

import pandas as pd
from utils import load_jsonl, dump_jsonl
import numpy as np

In [2]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [3]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [4]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [5]:
# df.groupby("label").count()

In [6]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [7]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2463 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [8]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,303,303,303
2. Normal,1403,1403,1403
3. Not respect,346,346,346


In [9]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [10]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


## Load Lexicons

In [11]:
from pythainlp.tokenize import word_tokenize

In [12]:
import json
with open("./PrivateSpace/thai-dictionary/RoyalInstituteDictionary/words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [13]:
lexicons_arr = load_jsonl("lexicons.jsonl")

Loaded 25573 records from lexicons.jsonl


In [14]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue
        
    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")
        
    w = word_tokenize(key)
    
    lexicons_keys[w[0]].append(key)
    
    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [15]:
from pythainlp.util import countthai

In [16]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords
    
def analyse_conv_per_person(texts):
    
    # Word Statistic
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    nlongword = 0
    ndict = 0
    for sent in words:
        ndict += sum([1 if w in thaidict_royal else 0 for w in sent])
        nlongword += sum([1 if len(w) > 7 else 0 for w in sent])
        nthai = sum([1 if countthai(w) > 50 else 0 for w in sent])
        
        
    
    uwords = set()
    for sent in words:
        uwords.update(sent)
    
    # Lexicon 
    lex = []
    for sidx, sent in enumerate(words):
        for widx, w in enumerate(sent):
            if w not in lexicons_keys:
                continue 
            
            s = "".join(sent[widx:])
            for l in lexicons_keys[w]:
                if not s.startswith(l):
                    continue

                lex.extend(lexicons[l])
#                 print(">>", w, l, lexicons[l])

    lexcat = {}
    for l in lex:
        if l not in lexcat:
            lexcat[l] = 0
        lexcat[l] += 1
    
    # Stylistic words
    nrepeat = 0
    for sidx, sent in enumerate(words):
        nrepeat += sum([1 if w=="rep" else 0 for w in sent])
    
    s = " ".join(texts)
    nemoji = emoji.emoji_count(s)
    

    return {
        "nsent": len(texts),
        "nword": sum([len(w) for w in words]),
        "ndict": ndict,
        "nunique": len(uwords),
        "nlongword": nlongword,
        "nrepeat": nrepeat,
        "nthai": nthai,
        "nemoji": nemoji,
        **lexcat
    }
  


def analyse_conversation(df):
    metrics = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        ms = analyse_conv_per_person(row["sys"])
        mu = analyse_conv_per_person(row["user"])
        metrics.append((ms, mu))
    
    return metrics

analyse_conv_per_person(["เมิงงงงงงงมันโง่เหมือนควายยยยยยยยย"]);

## Run Analysis

In [17]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [18]:
analysis_values = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

analysis_labels = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

In [19]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][0] = metrics
# analysis_labels["closeness"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][1] = metrics
# analysis_labels["closeness"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][2] = metrics
# analysis_labels["closeness"][2] = df["label"].values

In [20]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][0] = metrics
# analysis_labels["authority"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][1] = metrics
# analysis_labels["authority"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][2] = metrics
# analysis_labels["authority"][2] = df["label"].values

In [21]:
# save_obj_values("analysis_values.pkl", analysis_values)
# save_obj_values("analysis_labels.pkl", analysis_labels)

In [22]:
analysis_values = load_obj_values("analysis_values.pkl")
analysis_labels = load_obj_values("analysis_labels.pkl")

## Appendix C: Descriptive Analytics

In [25]:
metric_names = {
    "Word Statistics" : {
        "nsent": "Number of utterance",
        "nword": "Number of word",
    },
    "Lexicon Diversity" : {
        "nunique": "Vocabulary size",
        "nthai": "Thai words",
        "nlongword": "Long words",
        "ndict": "Dictionary words"
    },
    
    "Function Words": {

        "pronoun": "Pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",

        "particles": "Particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },
    
    "Sentiment-related": {
    
        "sentiment": "Sentiment words",
        "sentiment_positive": ">> Positive words",
        "sentiment_negative": ">> Negative words",
    },
    
    "Internet Lexicons": {
        "misspelling": "Spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Sematic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",

        "abbr": "Abbreviation",
        "slang": "Slang",
        "swear": "Swear words",
        "transliterated": "Transliteration",
    }
}

In [26]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

In [27]:
def print_lexi_stat(analysis_values, analysis_labels, factor, setting, print_labels):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    printed_text = ""
    rows = []
    for (ms, mu), l in zip(values, labels):
        for m in mu:
            if m in ["nsent", "nword"]:
                v = mu[m]
            else:
                v = mu[m]*100/mu["nword"]
                
            rows.append({
                "label": l,
                "metric": m,
                "value": v
            })
        
        if "particles_SARP" in mu:
            particles_notSARP = mu["particles"] - mu["particles_SARP"]
            rows.append({
                "label": l,
                "metric": "particles_notSARP",
                "value": particles_notSARP
            })
            
    df = pd.DataFrame(rows).groupby(["label", "metric"]).mean().reset_index()
    
    for g in metric_names:
        # print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{5}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for l in print_labels:
                row = df[(df["label"]==l) & (df["metric"]==m)]
#                 print(l, m)
#                 print(df)
                if len(row)!=0:
                    s += f"& {row['value'].values[0]:.2f} "
                else:
                    s += f"& - "

            s += "\\\\"
            # print(s)
            printed_text += "    "+s+"\n"
        
        printed_text += "    &  & &  & \\\\"+"\n"
        printed_text += "\hline"+"\n"
        
        # print("&  & &  & \\\\")
        # print("\hline")
    # print(printed_text)
    return df, printed_text

In [28]:
df, printed_text1 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 0, clse_print_labels)

In [29]:
df, printed_text2 = print_lexi_stat(analysis_values, analysis_labels, "authority", 0, auth_print_labels)

In [30]:
df, printed_text3 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 1, clse_print_labels)

In [31]:
df, printed_text4 = print_lexi_stat(analysis_values, analysis_labels, "authority", 1, auth_print_labels)

In [32]:
df, printed_text5 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 2, clse_print_labels)

In [33]:
# set(analysis_labels["authority"][2])

In [34]:
df, printed_text6 = print_lexi_stat(analysis_values, analysis_labels, "authority", 2, auth_print_labels)

In [35]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (printed_text1, printed_text2),
    (printed_text3, printed_text4),
    (printed_text5, printed_text6),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):
    
    printed_text += "\subsection{"+section+"}"+"\n\n"
    
    printed_text += '''\subsubsection{Closeness}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline

    Linguistic Features & Close & Know each other & Don't know each other &  Don't like each other\\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t1).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"

    printed_text += '''\clearpage
\subsubsection{Respect}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.16\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline

    Linguistic Features & Very respect & Respect & Normal &  Not respect\\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t2).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"
    printed_text += "\clearpage"
    printed_text += "\n\n"
    # break
    
# print(printed_text)

## Appendix D: Regression Analysis
https://medium.com/@wyess/demystifying-statistical-analysis-3-the-one-way-anova-expressed-in-linear-regression-99269e84edd9

In [37]:
# def closeness_to_vec1(label):
#     if label == "1. Close":
#         return {"x1": 1, "x2": 0}
#     elif label == "2. Know each other": ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == "3. Don't know each other": 
#         return {"x1": 0, "x2": 1}
#     elif label == "4. Don't like each other":
#         return None
#     else:
#         return None
    
# def authority_to_vec1(label):
#     if label == '0. Very respect':
#         return {"x1": 1, "x2": 0}
#     elif label == '1. Respect': ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == '2. Normal':
#         return {"x1": 0, "x2": 1}
#     elif label == '3. Not respect':
#         return None
#     else:
#         return None

def closeness_to_vec1(label):
    if label == "1. Close":
        return {"x1": 1, "x2": -1}
    elif label == "2. Know each other":
        return {"x1": 0, "x2": 2}
    elif label == "3. Don't know each other": 
        return {"x1": -1, "x2": -1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec1(label):
    if label == '0. Very respect':
        return {"x1": 1, "x2": -1}
    elif label == '1. Respect': 
        return {"x1": 0, "x2": 2}
    elif label == '2. Normal':
        return {"x1": -1, "x2": -1}
    elif label == '3. Not respect':
        return None
    else:
        return None

In [38]:
clse_coef_labels = ["b", "a1", "a2"]
auth_coef_labels = ["b", 'a1', 'a2']

In [39]:
def to_features(analysis_values, analysis_labels, factor, setting, to_vec_func, skips=[]):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    rows = []
    for (ms, mu), l in zip(values, labels):
        if l in skips:
            continue
            
        x = to_vec_func(l)
        if x is None:
            continue
        
        for m in mu:
            if m in ["nsent", "nword"]:
                v = mu[m]
            else:
                v = mu[m]*100/mu["nword"]
            
            
            rows.append({
                "metric": m,
                "value": v,
                **x
            })
            
        if "particles_SARP" in mu:
            particles_notSARP = mu["particles"] - mu["particles_SARP"]
            rows.append({
                "metric": "particles_notSARP",
                "value": particles_notSARP,
                **x
            })
            
    feats = pd.DataFrame(rows)  
    return feats

In [40]:
import statsmodels.api as sm
import numpy as np

def print_anova_test(feats, n=3):
    feat_names = feats["metric"].unique()
    
    feat_important = []
    for f in feat_names:
        if "cat:" in f:
            continue

        try:
            d = feats[feats["metric"]==f]
        #     print(f, len(d))
            x_columns = [f"x{i+1}" for i in range(n)]
            X = d[x_columns]
            X = sm.add_constant(X)
            Y = d["value"]
            model = sm.OLS(Y,X)
            results = model.fit()
            
            ncoef = n + 1
            
            t_test = results.t_test(np.identity(ncoef))
            f_test = results.f_test(np.identity(ncoef))
            o = {"feat": f, "f_value": f_test.pvalue}
            for i in range(ncoef):
                o[f"coef{i}"] = t_test.effect[i]
                
            for i in range(ncoef):
                o[f"pval{i}"] = t_test.pvalue[i]
                
            feat_important.append(o)
        except Exception as e:
            print("error", f, e)

    outputs = pd.DataFrame(feat_important)
    return outputs

In [41]:
# def print_weights(outputs, coef_labels, labels, base_category=None):
    
#     coefs = {}
#     non_coefs = {}
#     for i, label in enumerate(coef_labels):
#         if i==0:
#             continue
            
#         cond = outputs[f"pval{i}"] < 0.05
#         for _, row in outputs[cond].sort_values(f"coef{i}", ascending=False).iterrows():
#             coefs[(label, row["feat"])] = row[f"coef{i}"]
        
#         cond = outputs[f"pval{i}"] >= 0.05
#         for _, row in outputs[cond].iterrows():
#             non_coefs[(label, row["feat"])] = row[f"coef{i}"]

# #     print(coefs)
#     for g in metric_names:
#         print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
#         for m in metric_names[g]:
#             s = f"{metric_names[g][m]} "
#             for l in labels:
#                 if l == base_category:
#                     s += f"& * "
#                 elif (l, m) in coefs:
#                     val = coefs[(l, m)]
#                     s += "& \cellcolor{gray!25} "+f"{val:.2f} "
#                 elif (l, m) in non_coefs:
#                     val = non_coefs[(l, m)]
#                     s += f"& {val:.2f} "
#                 else:
#                     s += f"& - "

#             s += "\\\\"
#             print(s)
#         print("&  & &  & \\\\")
#         print("\hline")

# # gray!25 < gray!50 < gray!80

In [42]:
def print_weights(outputs):
    
    coefs = {}
    coef_labels = ["b", "a1", "a2"]
    for i, label in enumerate(coef_labels):
        for _, row in outputs.iterrows():
            coefs[(label, row["feat"])] = (row[f"coef{i}"], row[f"pval{i}"])

#     print(coefs)

    printed_text = ""
    for g in metric_names:
        # print("\multicolumn{4}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for l in coef_labels:                    
                val, pval = coefs[(l, m)]
                if l=="b":
                    s += f"& {val:.2f} "
                elif pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                else:
                    s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
        printed_text += "&  & &\\\\" + "\n"
        printed_text += "\hline" + "\n"
        # print("&  & &\\\\")
        # print("\hline")
    return printed_text

In [43]:
feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
outputs = print_anova_test(feats, n=2)
# print_weights(outputs, clse_coef_labels, clse_print_labels, "2. Know each other")
printed_text1 = print_weights(outputs)

In [44]:
feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=["3. Not respect"])
outputs = print_anova_test(feats, n=2)
# print_weights(outputs, auth_coef_labels, auth_print_labels, "1. Respect")
printed_text2 = print_weights(outputs)

In [45]:
# def closeness_to_vec2(label):
#     if label == "1. Close":
#         return {"x1": 1, "x2": 0}
#     elif label == "2. Know each other": 
#         return {"x1": 1, "x2": 0}
#     elif label == "3. Don't know each other": ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == "4. Don't like each other":
#         return {"x1": 0, "x2": 1}
#     else:
#         return None
    
# def authority_to_vec2(label):
#     if label == '0. Very respect':
#         return None
#     elif label == '1. Respect': 
#         return {"x1": 1, "x2": 0}
#     elif label == '2. Normal': ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == '3. Not respect':
#         return {"x1": 0, "x2": 1}
#     else:
#         return None

def closeness_to_vec2(label):
    if label == "1. Close":
        return {"x1": 1, "x2": -1}
    elif label == "2. Know each other": 
        return {"x1": 0, "x2": 2}
    elif label == "3. Don't know each other": ## Base category
        return {"x1": -1, "x2": 1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec2(label):
    if label == '0. Very respect':
        return None
    elif label == '1. Respect': 
        return {"x1": 1, "x2": -1}
    elif label == '2. Normal': ## Base category
        return {"x1": 0, "x2": 2}
    elif label == '3. Not respect':
        return {"x1": -1, "x2": -1}
    else:
        return None

In [46]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec2)
outputs = print_anova_test(feats, n=2)
# clse_coef_labels = ["b", "2. Know each other", "4. Don't like each other"]
# print_weights(outputs, clse_coef_labels, clse_print_labels, "3. Don't know each other")
printed_text3 = print_weights(outputs)

error particles_?? wrong shape for coefs


In [47]:
feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
outputs = print_anova_test(feats, n=2)
# auth2_coef_labels = ["b", '1. Respect', '3. Not respect']
# print_weights(outputs, auth2_coef_labels, auth_print_labels, "2. Normal")
printed_text4 = print_weights(outputs)

error particles_?? wrong shape for coefs


In [48]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec2, skips=[])
outputs = print_anova_test(feats, n=2)
# print_weights(outputs, clse_coef_labels, clse_print_labels, "2. Know each other")
printed_text5 = print_weights(outputs)

feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2, skips=[])
outputs = print_anova_test(feats, n=2)
# print_weights(outputs, auth_coef_labels, auth_print_labels, "1. Respect")
printed_text6 = print_weights(outputs)

/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [50]:
# sections = [
#     "Setting 1: Private Conversations with Self-Reported Labels",
#     "Setting 2: Public Conversations with Labels from 3rd Party ",
#     "Setting 3: Private Conversations with Labels from 3rd Party ",
# ]

# table_contents = [
#     (printed_text1, printed_text2),
#     (printed_text3, printed_text4),
#     (printed_text5, printed_text6),
# ]

# printed_text = ""
# for section, (t1, t2) in zip(sections, table_contents):
    
#     printed_text += "\subsection{"+section+"}"+"\n\n"
    
#     printed_text += '''\subsubsection{Closeness}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c
# }
#     \hline

#     Lexical Features & b & a1 & a2 \\\\
#     \hline
#     \endfirsthead

#     \endhead
#     '''
    
#     printed_text += ("\n"+t1).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"

#     printed_text += '''\clearpage
# \subsubsection{Respect}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c
# }
#     \hline

#     Lexical Features & b & a1 & a2 \\\\
#     \hline
#     \endfirsthead

#     \endhead
#     '''
    
#     printed_text += ("\n"+t2).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"
    
#     printed_text += "\clearpage"
#     printed_text += "\n\n"
#     # break
    
# print(printed_text)

## Appendix D2

In [97]:
def closeness_to_vec_effect(label):
    if label == "1. Close":
        return {"x1": 1, "x2": 0}
    elif label == "2. Know each other": ## Base category
        return {"x1": -1, "x2": -1}
    elif label == "3. Don't know each other": 
        return {"x1": 0, "x2": 1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec_effect(label):
    if label == '0. Very respect':
        return {"x1": 1, "x2": 0}
    elif label == '1. Respect': 
        return {"x1": 0, "x2": 1}
    elif label == '2. Normal':    ## Base category
        return {"x1": -1, "x2": -1}
    elif label == '3. Not respect':
        # return None
        return {"x1": 1, "x2": 0}
    else:
        return None

In [137]:
def print_weights_effect(outputs, coef_labels):
    coefs = {}
    for i, label in coef_labels.items():
        if i is None:
            continue
            
        for _, row in outputs.iterrows():
            coefs[(label, row["feat"])] = (row[f"coef{i}"], row[f"pval{i}"])

    printed_text = ""
    for g in metric_names:
        # print("\multicolumn{4}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for i, l in coef_labels.items():                    
                if i is None:
                    s += f"& * "
                    continue
                
                val, pval = coefs[(l, m)]
                if l=="Grand Mean":
                    s += f"& {val:.2f} "
                elif pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                else:
                    s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
        printed_text += "&  & &\\\\" + "\n"
        printed_text += "\hline" + "\n"
        # print("&  & &\\\\")
        # print("\hline")
    return printed_text, coef_labels

feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec_effect, skips=[])
outputs = print_anova_test(feats, n=2)
# ["Grand Mean", "0. Very respect", "1. Respect", "", "3. Not respect"], "2. Normal"

coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text1 = print_weights_effect(outputs, coef_labels)

feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec_effect, skips=["3. Not respect"])
outputs = print_anova_test(feats, n=2)

coef_labels = {0:"Grand Mean", 1: "Very respect", 2: "Respect", None: "Normal"}
printed_text2 = print_weights_effect(outputs, coef_labels)

In [138]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec_effect, skips=[])
outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text3 = print_weights_effect(outputs, coef_labels)


feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec_effect, skips=["0. Very respect"])
outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", None: "Normal", 2: "Respect", 1: "Not respect"}
printed_text4 = print_weights_effect(outputs, coef_labels)

error particles_?? wrong shape for coefs
error particles_?? wrong shape for coefs


In [139]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec_effect, skips=[])
outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text5 = print_weights_effect(outputs, coef_labels)

feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec_effect, skips=["0. Very respect"])
outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", None: "Normal", 2: "Respect", 1: "Not respect"}
printed_text6 = print_weights_effect(outputs, coef_labels)

/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [140]:
# feats[feats["metric"]=="nsent"].groupby(["x1", "x2"]).count()

In [144]:
# sections = [
#     "Setting 1: Private Conversations with Self-Reported Labels",
#     "Setting 2: Public Conversations with Labels from 3rd Party ",
#     "Setting 3: Private Conversations with Labels from 3rd Party ",
# ]

# table_contents = [
#     (printed_text1, printed_text2),
#     (printed_text3, printed_text4),
#     (printed_text5, printed_text6),
# ]

# # columns = 
# # (["Grand Mean", "1. Close", "2. Know each other", "3. Don't know each other"], "2. Know each other")

# # ["Grand Mean", "0. Very respect", "1. Respect", "2. Normal", "3. Not respect"], "2. Normal"

# printed_text = ""
# for section, (v1, v2) in zip(sections, table_contents):
    
#     printed_text += "\subsection{"+section+"}"+"\n\n"
    
#     printed_text += '''\subsubsection{Closeness}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
# }
#     \hline
#     '''
    
#     t1, col1 = v1
#     printed_text += "Lexical Features & " + " & ".join(col1.values()) + "\\\\"
        
#     printed_text +='''
#     \hline
#     \endfirsthead

#     \endhead
#     '''

    
#     printed_text += ("\n"+t1).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"

    
#     printed_text += '''\clearpage
# \subsubsection{Respect}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
#     p{\dimexpr 0.15\linewidth-2\\tabcolsep}
# }
#     \hline
#     '''

#     t2, col2 = v2
    
#     printed_text += "Lexical Features & " + " & ".join(col2.values()) + "\\\\"
        
#     printed_text +='''
#     \hline
#     \endfirsthead

#     \endhead
#     '''
    
#     printed_text += ("\n"+t2).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"
    
#     printed_text += "\clearpage"
#     printed_text += "\n\n"
#     # break
    
# print(printed_text)

## Appendix E: Linear Weights

In [209]:
# from sklearn import datasets, linear_model
# from sklearn.linear_model import LinearRegression
# import statsmodels.api as sm
# from scipy import stats

In [252]:
# def to_linear_features(analysis_values, analysis_labels, factor, setting):
#     values = analysis_values[factor][setting]
#     labels = analysis_labels[factor][setting]
#     assert(len(values)==len(labels))
    
#     rows = []
#     for (ms, mu), l in zip(values, labels):
        
#         o = {"label": l, "bias": 1}
#         for m in mu:
#             if m in ["nsent", "nword"]:
#                 v = mu[m]
#             else:
#                 v = mu[m]*100/mu["nword"]
            
#             o[m] = v
#         rows.append(o)
        
#     feats = pd.DataFrame(rows).fillna(0)  
#     return feats



# def run_linear_model(feats, skips=[]):
#     col = []
#     for g in metric_names:
#         for m in metric_names[g]:
#             if m in ["nsent", "nword"]:
#                 continue
                
#             col.append(m)
    
#     X = feats[col].values

#     saved_results = {}
#     for label in labels:
#         if label in skips:
#             continue
        
#         print(label)
#         Y = (feats["label"]==label).astype(int)
#         model = sm.Logit(Y, X)
#         result = model.fit()
# #         print(result.summary())
#         ncoef = len(col)
#         t_test = result.t_test(np.identity(ncoef))
        
# #         print(t_test.tvalue) #Z
# #         print(t_test.pvalue) #p-value
# #         print(t_test.effect) #coef

#         for f, coef, pvalue in zip(feats, t_test.effect, t_test.pvalue):
#             saved_results[(label, f)] = (coef, pvalue)
#     return saved_results

# def print_linear_weights(weights, labels, skips=[]):
#     for g in metric_names:
#         print("\multicolumn{"+str(len(labels)+1)+"}{l}{\\textit{"+g+"}} \\\\")
#         for m in metric_names[g]:
#             if m in ["nwordputterance", "nsent", "nword"]:
#                 continue
                
#             s = f"{metric_names[g][m]} "
#             for l in labels:
#                 if l in skips:
#                     s += f"& - "
#                     continue
                
#                 if (l, m) not in weights:
#                     s += f"& - "
#                     continue
                    
#                 val, pvalue = weights[(l, m)]
#                 if pvalue < 0.05:
#                     s += "& \cellcolor{gray!25} "+f"{val:.3f} "
#                 else:
#                     s += f"& {val:.3f} "

#             s += "\\\\"
#             print(s)
#         print("&  & &  & \\\\")
#         print("\hline")
        
#     return weights

In [251]:
# feats = to_linear_features(analysis_values, analysis_labels, "closeness", 0)
# weights = run_linear_model(feats, skips=["4. Don't like each other"])
# print_linear_weights(weights, clse_print_labels, skips=["4. Don't like each other"]);

In [250]:
# feats = to_linear_features(analysis_values, analysis_labels, "authority", 0)
# weights = run_linear_model(feats, skips=["3. Not respect"])
# print_linear_weights(weights, auth_print_labels, skips=["3. Not respect"]);